In [ ]:
!git clone https://github.com/JDEA-Quantum-Lab/TeD-Q.git

In [ ]:
!rm -rf TeD-Q

In [ ]:

!pip install numpy torch jax jaxlib matplotlib panel jupyterlab ipywidgets toolz ray


In [ ]:
!pip uninstall tedq

In [ ]:
!pip install -e .

In [ ]:
!git clone https://github.com/JDEA-Quantum-Lab/JDtensorPath.git

In [ ]:
!cd ../JDtensorPath
!pip install tedq kahypar==1.1.6
!pip install -e .

In [ ]:
import tedq as qai
from matplotlib import pyplot as plt
import numpy as np
import scipy
import networkx as nx
import copy

In [ ]:
!pip install qiskit

In [ ]:
!pip uninstall tornado -y

In [ ]:
!pip install jinja2==3.0.1

In [ ]:
!yes | pip install tornado==5.1.0

In [ ]:
!pip install networkx

In [ ]:
qubit_number = 4
qubits = range(qubit_number)
ising_graph = nx.cycle_graph(qubit_number)

print(f"Edges: {ising_graph.edges}")
nx.draw(ising_graph)

In [ ]:
target_weights = [0.56, 1.24, 1.67, -0.79]
target_bias = [-1.44, -1.43, 1.18, -0.93]
def create_hamiltonian_matrix(n_qubits, graph, weights, bias):

    full_matrix = np.zeros((2 ** n_qubits, 2 ** n_qubits))

    # Creates the interaction component of the Hamiltonian
    for i, edge in enumerate(graph.edges):
        interaction_term = 1
        for qubit in range(0, n_qubits):
            if qubit in edge:
                Z = np.array([[ 1,  0],[ 0, -1]])
                interaction_term = np.kron(interaction_term, Z)
            else:
                interaction_term = np.kron(interaction_term, np.identity(2))
        full_matrix += weights[i] * interaction_term

    # Creates the bias components of the matrix
    for i in range(0, n_qubits):
        z_term = x_term = 1
        for j in range(0, n_qubits):
            if j == i:
                Z = np.array([[ 1,  0],[ 0, -1]])
                X = np.array([[0, 1],
       [1, 0]])
                z_term = np.kron(z_term, Z)
                x_term = np.kron(x_term, X)
            else:
                z_term = np.kron(z_term, np.identity(2))
                x_term = np.kron(x_term, np.identity(2))
        full_matrix += bias[i] * z_term + x_term

    return full_matrix


# Prints a visual representation of the Hamiltonian matrix
ham_matrix = create_hamiltonian_matrix(qubit_number, ising_graph, target_weights, target_bias)
plt.matshow(ham_matrix, cmap="hot")
plt.show()

In [ ]:
low_energy_state = [
    (-0.054661080280306085 + 0.016713907320174026j),
    (0.12290003656489545 - 0.03758500591109822j),
    (0.3649337966440005 - 0.11158863596657455j),
    (-0.8205175732627094 + 0.25093231967092877j),
    (0.010369790825776609 - 0.0031706387262686003j),
    (-0.02331544978544721 + 0.007129899300113728j),
    (-0.06923183949694546 + 0.0211684344103713j),
    (0.15566094863283836 - 0.04760201916285508j),
    (0.014520590919500158 - 0.004441887836078486j),
    (-0.032648113364535575 + 0.009988590222879195j),
    (-0.09694382811137187 + 0.02965579457620536j),
    (0.21796861485652747 - 0.06668776658411019j),
    (-0.0027547112135013247 + 0.0008426289322652901j),
    (0.006193695872468649 - 0.0018948418969390599j),
    (0.018391279795405405 - 0.005625722994009138j),
    (-0.041350974715649635 + 0.012650711602265649j),
]
res = np.vdot(low_energy_state, (ham_matrix @ low_energy_state))
energy_exp = np.real_if_close(res)
print(f"Energy Expectation: {energy_exp}")


ground_state_energy = np.real_if_close(min(np.linalg.eig(ham_matrix)[0]))
print(f"Ground State Energy: {ground_state_energy}")

In [ ]:
np.linalg.eig(ham_matrix)

In [ ]:
!pip install pennylane

In [ ]:
import pennylane as qml

In [ ]:
def state_evolve(hamiltonian, qubits, time):

    U = scipy.linalg.expm(-1j * hamiltonian * time)
    qai.QubitUnitary(U, wires=qubits)

In [ ]:
def qgrnn_layer(weights, bias, qubits, graph, trotter_step):
    print(1.1)
    # Applies a layer of RZZ gates (based on a graph)
    for i, edge in enumerate(graph.edges):
        qml.MultiRZ(2 * weights[i] * trotter_step, wires=(edge[0], edge[1]))
       # qai.RZ(2 *  weights[i], qubits=[edge[0]])
       # qai.RZ(trotter_step, qubits=[edge[1]])
    print(1.2)
    # Applies a layer of RZ gates
    for i, qubit in enumerate(qubits):
        print(1.21)
        qai.RZ(2 * bias[i] * trotter_step, qubits=[qubit])
    print(1.3)
    # Applies a layer of RX gates
    for qubit in qubits:
        qai.RX(2 * trotter_step, qubits=[qubit])

In [ ]:
for i, edge in enumerate(new_ising_graph.edges):
  print(edge[0], edge[1])

In [ ]:
def swap_test(control, register1, register2):
    #print(2.1)
    qai.Hadamard(qubits=[control])
    for reg1_qubit, reg2_qubit in zip(register1, register2):
        #print(2.2)
        qai.CSWAP(qubits=(control, reg1_qubit, reg2_qubit))
    #print(2.3)
    qai.Hadamard(qubits=[control])

In [ ]:
# Defines some fixed values
import torch
reg1 = tuple(range(qubit_number))  # First qubit register
reg2 = tuple(range(qubit_number, 2 * qubit_number))  # Second qubit register

control = 2 * qubit_number  # Index of control qubit
trotter_step = torch.tensor(0.01)  # Trotter step size

# Defines the interaction graph for the new qubit system

new_ising_graph = nx.complete_graph(reg2)

print(f"Edges: {new_ising_graph.edges}")
nx.draw(new_ising_graph)

In [ ]:

#torch.nn.Parameter(torch.tensor(rng.random(size=len(new_ising_graph.edges))), requires_grad=True) - 0.5
#bias = torch.nn.Parameter(torch.tensor(rng.random(size=qubit_number)), requires_grad=True) - 0.5

def qgrnn(weights, bias, time):
  
    # Prepares the low energy state in the two registers
    qml.QubitStateVector(np.kron(low_energy_state, low_energy_state), wires=reg1 + reg2)
    #qai.QubitStateVector(np.kron(low_energy_state, low_energy_state), wires=reg1 + reg2)

    # Evolves the first qubit register with the time-evolution circuit to
    # prepare a piece of quantum data
    # re_all = np.kron(low_energy_state, low_energy_state)
    # U = scipy.linalg.expm(-1j * ham_matrix * time)
    # re1 = np.matmul(U, re_all[reg1])
    # re2 = re_all[reg2]
   # print(0)
    state_evolve(ham_matrix, reg1, time)
   # print(1)
    #time = 1
    # Applies the QGRNN layers to the second qubit register
    depth = time / trotter_step  # P = t/Delta
    for _ in range(0, int(depth)):
        qgrnn_layer(weights, bias, reg2, new_ising_graph, trotter_step)
    print(2)
    # Applies the SWAP test between the registers
    swap_test(control, reg1, reg2)
    print(3)
    # Returns the results of the SWAP test
    #Re = qai.PauliZ(qubits = [control])
    print(4)
    #print(Re)
    #import pdb
    #pdb.set_trace()

   # qai.measurement.expval(qai.PauliZ(qubits=[control]))#[qai.expval(qai.PauliZ(qubits=[position])) for position in range(control)] 
    
    print(6)
    return qai.measurement.expval(qai.PauliZ(qubits=[control]))#qai.measurement.expval(qai.PauliZ(qubits=[1]))

In [ ]:
N = 15  # The number of pieces of quantum data that are used for each step
max_time = 0.1  # The maximum value of time that can be used for quantum data

In [ ]:
rng = np.random.default_rng(seed=42)
def cost_function(weight_params, bias_params):

    # Randomly samples times at which the QGRNN runs
    times_sampled = rng.random(size=N) * max_time
    print(7)
    # Cycles through each of the sampled times and calculates the cost
    total_cost = 0
    for dt in times_sampled:
        print(8)
        result = qgrnn(weight_params, bias_params, time=dt)
        print(9)
        total_cost += -1 * result

    return total_cost / N

In [ ]:
def ted_state_evolve(hamiltonian, qubits, time):

    U = scipy.linalg.expm(-1j * hamiltonian * time)
    #print(qubits)
    qai.Unitary(U, qubits=list(qubits))

In [ ]:
def ted_qgrnn_layer(weights, bias, qubits, graph, trotter_step):
    print(1.1)
    # Applies a layer of RZZ gates (based on a graph)
    #for i, edge in enumerate(graph.edges):
    #    qml.MultiRZ(2 * weights[i] * trotter_step, qubits=[edge[0], edge[1]])
    #   # qai.RZ(2 *  weights[i], qubits=[edge[0]])
    #   # qai.RZ(trotter_step, qubits=[edge[1]])
    #print(1.2)
    # Applies a layer of RZ gates
    for i, qubit in enumerate(qubits):
        #print(1.21)
        #print(qubit)
        qai.RZ(2 * bias[i] * trotter_step, qubits=[qubit])
    #print(1.3)
    # Applies a layer of RX gates
    for qubit in qubits:
        qai.RX(2 * trotter_step, qubits=[qubit])

In [ ]:


def ted_qgrnn(weights, bias, time):
  
    # Prepares the low energy state in the two registers
    #qml.QubitStateVector(np.kron(low_energy_state, low_energy_state), wires=reg1 + reg2)
    #qai.IintStateVector(np.kron(low_energy_state, low_energy_state))

    # Evolves the first qubit register with the time-evolution circuit to
    # prepare a piece of quantum data
    # re_all = np.kron(low_energy_state, low_energy_state)
    # U = scipy.linalg.expm(-1j * ham_matrix * time)
    # re1 = np.matmul(U, re_all[reg1])
    # re2 = re_all[reg2]
   # print(0)
    ted_state_evolve(ham_matrix, reg1, time)
   # print(1)
    #time = 1
    # Applies the QGRNN layers to the second qubit register
    depth = time / trotter_step  # P = t/Delta
    for _ in range(0, int(depth)):
        ted_qgrnn_layer(weights, bias, reg2, new_ising_graph, trotter_step)
    #print(2)
    # Applies the SWAP test between the registers
    #swap_test(control, reg1, reg2)
    #print(3)
    # Returns the results of the SWAP test
    #Re = qai.PauliZ(qubits = [control])
    #print(4)
    #print(Re)
    #import pdb
    #pdb.set_trace()

   # qai.measurement.expval(qai.PauliZ(qubits=[control]))#[qai.expval(qai.PauliZ(qubits=[position])) for position in range(control)] 
    
    #print(6)
    return qai.measurement.expval(qai.PauliZ(qubits=[control]))#qai.measurement.expval(qai.PauliZ(qubits=[1]))

In [ ]:
import torch
# Defines the new device
#qgrnn_dev = qai.device("default.qubit", wires=2 * qubit_number + 1)

# Defines the new QNode
#qgrnn_qnode = qai.QNode(qgrnn)
rng = np.random.default_rng(seed=42)
steps = 300

#optimizer =  qai.GradientDescentOptimizer(stepsize=0.5, interface="pytorch")
weights = torch.rand(len(new_ising_graph.edges)) - 0.5
bias = torch.rand(qubit_number) - 0.5
print(weights,bias)

initial_weights = copy.copy(weights)
initial_bias = copy.copy(bias)
#weights, bias, reg2, new_ising_graph, trotter_step
#circuit = qai.Circuit(qgrnn_layer, qubit_number, weights, bias, reg2, new_ising_graph, trotter_step  )
times_sampled = rng.random(size=1) * max_time
circuit = qai.Circuit(ted_qgrnn, 9, weights, bias, times_sampled)#, weights, bias, torch.rand(qubit_number))
print(6)
my_compilecircuit = circuit.compilecircuit(backend="pytorch" )

In [ ]:
def dressed_layer(parameters, weights, bias, qubits, graph, trotter_step):

    for i, qubit in enumerate(qubits):
        parameters.append(2 * bias[i] * trotter_step)

    for qubit in qubits:
        parameters.append(2 * trotter_step)

In [ ]:
def dressed_compilecircuit(weights, bias, time):
    parameters = []
    depth = time / trotter_step  # P = t/Delta
    for _ in range(0, int(depth)):
        dressed_layer(parameters, weights, bias, reg2, new_ising_graph, trotter_step)
    
    print("len of parameters:",  len(parameters))
    parameters = torch.stack(parameters)
    
    circuit = qai.Circuit(ted_qgrnn, 9, weights, bias, time)#, weights, bias, torch.rand(qubit_number))
    my_compilecircuit = circuit.compilecircuit(backend="pytorch" )
    
    return my_compilecircuit(parameters)
    

In [ ]:
rng = np.random.default_rng(seed=42)
N = 1
def ted_cost_function(weight_params, bias_params):

    # Randomly samples times at which the QGRNN runs
    times_sampled = rng.random(size=N) * max_time
    print(7)
    # Cycles through each of the sampled times and calculates the cost
    total_cost = 0
    for dt in times_sampled:
        print('cyc1')
        result = dressed_compilecircuit(weight_params, bias_params, time=dt)
        print(result)
        print('cyc2')
        total_cost += -1 * result

    return total_cost / N

In [ ]:
from torch import optim
#bias = bias.cuda()
#weights = weights.cuda()
weights.requires_grad = True
bias.requires_grad = True

optimizer = optim.Adam([weights, bias], lr=0.01)
for i in range(0, steps):
    optimizer.zero_grad()
    #(weights, bias), cost = optimizer.step(cost_function)
    loss = ted_cost_function(weights, bias)
    loss.backward()
    optimizer.step()
    # Prints the value of the cost function
    if i % 5 == 0:
        print(f"Cost at Step {i}: {loss}")
        print(f"Weights at Step {i}: {weights}")
        print(f"Bias at Step {i}: {bias}")
        print("---------------------------------------------")